# PREPROCESAMIENTO INTEGRAL PARA ANÁLISIS DE MARKETING: SHOWZ

Este notebook documenta el proceso de extracción, limpieza y optimización de los registros de servidor, pedidos y gastos de marketing de la empresa Showz.

### Objetivos Técnicos:
* **Diagnóstico de Calidad:** Identificación de valores ausentes, duplicados e inconsistencias de tipos.
* **Optimización de Memoria:** Implementación de Pandas 3.0 con motor PyArrow para un manejo eficiente de grandes volúmenes de datos.
* **Persistencia de Tipado:** Los datos resultantes se exportarán en formato `Parquet`, garantizando que las jerarquías de tipos (especialmente fechas y categorías) se mantengan intactas para el análisis posterior.

**Nota:** Aunque este proceso se presenta aquí de forma visual para fines de auditoría, la lógica está modularizada en la carpeta src/, permitiendo su ejecución automatizada mediante scripts de Python. Para la interpretación estratégica y conclusiones de negocio, por favor consulte:
`notebooks/marketing_report_showz.ipynb`.

---
---

## 1. Carga y Diagnóstico Inicial de Datos

Para garantizar un flujo de trabajo modular y reproducible, utilizaremos el módulo interno src.data_loader. Este paso nos permite auditar la calidad de la información cruda (raw data) antes de cualquier transformación.

Realizaremos una inspección de:
* **Integridad Estructural:** Dimensiones y tipos de datos iniciales.
* **Calidad de Datos:** Identificación de valores nulos y duplicados explícitos.
* **Consistencia:** Verificación visual de los primeros registros para detectar anomalías de formato.

In [1]:
from IPython.display import display, Markdown
import sys 
from pathlib import Path
# Añadimos la raíz del proyecto al path para que Python encuentre src/
sys.path.append(str(Path.cwd().parent))
# Importamos src/data_loader.py
from src.data_loader import load_raw_data, get_data_diagnostics, standardize_dataframe
# Cargamos los datos raw
visits, orders, costs = load_raw_data()
# Obtenemos el diagnóstico de los datos
get_data_diagnostics(visits, "Logs de Visitas")
get_data_diagnostics(orders, "Registros de Pedidos")
get_data_diagnostics(costs, "Estadísticas de Gastos")

## Diagnóstico de: `Logs de Visitas`

**Dimensiones:** 359,400 filas × 5 columnas &ensp;|&ensp; **Memoria:** 28.95 MB

,Column,Type,Non-Null,Nulls,% Nulls,Unique
0,Device,str,359400,0,0.0,2
1,End Ts,str,359400,0,0.0,224760
2,Source Id,int64,359400,0,0.0,9
3,Start Ts,str,359400,0,0.0,224303
4,Uid,uint64,359400,0,0.0,228169


**Filas duplicadas (completas):** 0 (0.00%)

**Resumen numérico:**

...

**Filas duplicadas (completas):** 0 (0.00%)

**Resumen numérico:**

,source_id,costs
count,2542.00,2542.00
mean,4.86,129.48
std,3.18,156.30
min,1.00,0.54
25%,2.00,21.94
50%,4.00,77.30
75%,9.00,170.06
max,10.00,1788.28


**Muestra (3 filas):**

,source_id,dt,costs
0,1,2017-06-01,75.20
1,1,2017-06-02,62.25
2,1,2017-06-03,36.53


### Transformaciones Técnicas Identificadas

Tras la inspección, se determinan los siguientes ajustes estructurales necesarios para estandarizar el dataset:
* **Normalización de Nombres:** Aplicar snake_case a todas las columnas (ej. Start Ts $\rightarrow$ start_ts, Revenue $\rightarrow$ revenue).
* **Tipado con PyArrow (Pandas 3.0):** 
    * **Fechas:** Convertir columnas temporales a timestamp[ns][pyarrow].
    * **IDs:** Optimizar `uid` y `source_id` a enteros de memoria reducida.
    * **Categorías:** Transformar la columna `device` a tipo **category** para reducir la carga en memoria.
* **Consistencia de Precios:** Asegurar que `revenue` sea tratado como flotante de alta precisión.

In [2]:
# ── Mapeos de tipos eficientes por dataset ──────────────────────────

visits_dtypes = {
    "device":    "category",
    "end_ts":    "timestamp[ns][pyarrow]",
    "source_id": "int32[pyarrow]",
    "start_ts":  "timestamp[ns][pyarrow]",
    "uid":       "uint64[pyarrow]",
}

orders_dtypes = {
    "buy_ts":  "timestamp[ns][pyarrow]",
    "revenue": "double[pyarrow]",
    "uid":     "uint64[pyarrow]",
}

costs_dtypes = {
    "source_id": "int32[pyarrow]",
    "dt":        "timestamp[ns][pyarrow]",
    "costs":     "double[pyarrow]",
}

# ── Aplicar estandarización ─────────────────────────────────────────

visits = standardize_dataframe(visits, dtypes=visits_dtypes)
orders = standardize_dataframe(orders, dtypes=orders_dtypes)
costs  = standardize_dataframe(costs, dtypes=costs_dtypes)

# Verificación rápida
for name, df in [("visits", visits), ("orders", orders), ("costs", costs)]:
    print(f"\n{'─'*15} {name} {'─'*15}")
    print(df.dtypes)


─────────────── visits ───────────────
device                     category
end_ts       timestamp[ns][pyarrow]
source_id            int32[pyarrow]
start_ts     timestamp[ns][pyarrow]
uid                 uint64[pyarrow]
dtype: object

─────────────── orders ───────────────
buy_ts     timestamp[ns][pyarrow]
revenue           double[pyarrow]
uid               uint64[pyarrow]
dtype: object

─────────────── costs ───────────────
source_id            int32[pyarrow]
dt           timestamp[ns][pyarrow]
costs               double[pyarrow]
dtype: object


---

## 2. Validación de Integridad de Negocio

Antes de proceder con el análisis, es imperativo garantizar la coherencia lógica de los datos. No basta con que los tipos sean correctos; los datos deben hacer sentido comercial. Evaluaremos:
* **Validación Temporal:** ¿Existen sesiones donde `end_ts` sea anterior a `start_ts`?
* **Análisis de Outliers:** Identificación de pedidos con **revenue** inusualmente alto o valores cero que puedan sesgar el LTV.
* **Duplicados Implícitos:** Verificación de registros idénticos en logs de visitas (mismo usuario, dispositivo y marca de tiempo).

In [4]:
# Validación temporal: end_ts nunca debe ser anterior a start_ts
invalid_sessions = visits.query("end_ts < start_ts")
display(Markdown(
    f"**Sesiones con `end_ts` < `start_ts`:** {len(invalid_sessions):,} "
    f"({len(invalid_sessions) / len(visits) * 100:.2f}%)"
))
if len(invalid_sessions) > 0:
    display(invalid_sessions.head())

# Outliers en revenue (método IQR) ────────────────────────────
q1 = orders["revenue"].quantile(0.25)
q3 = orders["revenue"].quantile(0.75)
iqr = q3 - q1
lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr

outliers = orders.query("revenue < @lower or revenue > @upper")
zero_rev = orders.query("revenue == 0")

display(Markdown(
    f"**Outliers en revenue (IQR):** {len(outliers):,} "
    f"({len(outliers) / len(orders) * 100:.2f}%) &ensp;|&ensp; "
    f"**Rango aceptable:** [{lower:.2f}, {upper:.2f}]"
))
display(Markdown(f"**Pedidos con revenue = 0:**  {len(zero_rev):,}"))
display(outliers["revenue"].describe().round(2))

# Duplicados implícitos en visits (uid + device + start_ts) ───
dup_cols = ["uid", "device", "start_ts"]
implicit_dups = visits.duplicated(subset=dup_cols, keep=False)
n_dups = implicit_dups.sum()

display(Markdown(
    f"**Duplicados implícitos** (`uid` + `device` + `start_ts`): {n_dups:,} "
    f"({n_dups / len(visits) * 100:.2f}%)"
))
if n_dups > 0:
    display(visits[implicit_dups].sort_values(dup_cols).head(10))

**Sesiones con `end_ts` < `start_ts`:** 2 (0.00%)

,device,end_ts,source_id,start_ts,uid
4181,desktop,2018-03-25 03:18:00,3,2018-03-25 03:50:00,13092152539246794986
177972,desktop,2018-03-25 03:09:00,9,2018-03-25 03:55:00,4621202742905035453


**Outliers en revenue (IQR):** 3,990 (7.91%) &ensp;|&ensp; **Rango aceptable:** [-4.29, 10.39]

**Pedidos con revenue = 0:**  51

count     3990.0
mean       28.07
std        73.31
min        10.45
25%        12.22
50%        15.89
75%        23.83
max      2633.28
Name: revenue, dtype: double[pyarrow]

**Duplicados implícitos** (`uid` + `device` + `start_ts`): 2 (0.00%)

,device,end_ts,source_id,start_ts,uid
44993,touch,2018-03-16 08:57:00,1,2018-03-16 08:55:00,1981020429381477763
47067,touch,2018-03-16 08:55:00,1,2018-03-16 08:55:00,1981020429381477763


### Resultados de Validación de Integridad

Al analizar la consistencia, se han tomado las siguientes decisiones estratégicas para el tratamiento de los datos:
* **Remoción de Ruido Técnico:** Se eliminan las sesiones con tiempos inconsistentes (end_ts < start_ts) y duplicados implícitos. Su impacto en el volumen de datos es nulo (< 0.01%), pero su eliminación previene sesgos en métricas de duración y frecuencia.
* **Preservación de Outliers:** Se decide mantener todos los registros de revenue elevados. En un modelo de venta de entradas, los compradores de alto ticket son reales y críticos para un cálculo preciso del LTV y ROMI.
* **Inclusión de Conversiones Zero-Revenue:** Se mantienen los pedidos con valor 0. Representan conversiones reales (cortesías, promociones o errores de pago) y son esenciales para no subestimar el costo de adquisición (CAC).

---

## 3. Consolidación y Persistencia: Capa "Processed"

Tras validar la integridad de los datos, procedo a la ejecución de los filtros finales y la exportación de los datasets. Este paso representa el cierre del pipeline de ingeniería, donde los datos transitan definitivamente de su estado Raw a la capa Processed.

* **Ejecución de Criterios de Integridad:** Se aplican los filtros para remover el ruido técnico identificado (sesiones inconsistentes y duplicados), garantizando que solo la información válida pase a la fase de análisis.
* **Preservación de Esquema:** Aunque los datos ya operan bajo el motor de PyArrow, la persistencia en formato Parquet actúa como una "caja fuerte", almacenando los metadatos de tipado (fechas, categorías e identificadores optimizados) de forma nativa.
* **Eficiencia de Almacenamiento:** El uso de archivos Parquet permite una compresión superior y lecturas columnares ultra rápidas, eliminando la necesidad de re-procesar los tipos de datos al iniciar el informe de negocio.

In [ ]:
n_before = len(visits)

# Sesiones con end_ts < start_ts
visits = visits.query("end_ts >= start_ts")

# Duplicados implícitos (uid + device + start_ts)
visits = visits.drop_duplicates(subset=["uid", "device", "start_ts"])

display(Markdown(
    f"**visits:** {n_before:,} → {len(visits):,} "
    f"(−{n_before - len(visits):,} filas removidas)"
))

**visits:** 359,400 → 359,397 (−3 filas removidas)

In [7]:
# data/processed -> parquet
processed = Path.cwd().parent / "data" / "processed"
processed.mkdir(parents=True, exist_ok=True)

visits.to_parquet(processed / "visits.parquet")
orders.to_parquet(processed / "orders.parquet")
costs.to_parquet(processed / "costs.parquet")

print(f"Exportados a {processed}:")
for f in sorted(processed.glob("*.parquet")):
    print(f"  {f.name} — {f.stat().st_size / 1024:.0f} KB")

Exportados a c:\Portafolio_Data_Analyst\showz_marketing_roi\data\processed:
  costs.parquet — 22 KB
  orders.parquet — 865 KB
  visits.parquet — 10432 KB


Pipeline de datos finalizado. Los activos están optimizados, validados y listos para la fase de análisis estratégico en: `notebooks/marketing_report_showz.ipynb`.